# Lexicon Enrichment

In [1]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

24/12/14 12:14:22 WARN Utils: Your hostname, JORDAN. resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/12/14 12:14:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/12/14 12:14:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## User-Defined Function

`def findAntonym()`: to find the antonyms list of a word.

In [3]:
from python_script.antonym_finder import findAntonym

## Reorder the Columns

In [4]:
df = spark.read.csv('lexicon_data/processed_data.csv', header=True)
df = df.select('vocab', 'word_length', 'part_of_speech', 'definitions', 'sentences_eg', 'derived_words', 'synonyms')
df.show()

+-----------+-----------+--------------+--------------------+--------------------+--------------------+--------------------+
|      vocab|word_length|part_of_speech|         definitions|        sentences_eg|       derived_words|            synonyms|
+-----------+-----------+--------------+--------------------+--------------------+--------------------+--------------------+
|     sahaja|          6|    kata tugas|1. yg satu-satuny...|1. tuan sahaja yg...|bersahaja;bersaha...|satu-satunya;cuma...|
|      kerja|          5|     kata nama|1. usaha (kegiata...|1. beberapa mingg...|sekerja;bekerja;m...|khidmat;pekerjaan...|
|       kaum|          4|     kata nama|1. puak, suku ban...|1. banyak kaum or...|   berkaum;perkauman|suku;kabilah;taef...|
|      jelas|          5|      adjektif|1. terang (perkat...|1. suara itu tida...|berjelas-jelas;me...|terang;nyata;keta...|
|      waktu|          5|     kata nama|1. rangkaian atau...|1. secara rohania...|sewaktu;sewaktu-w...|masa;jangka masa;...|


## Insert Antonyms for each Rows (if any)

In [5]:
append_antonym_udf = udf(findAntonym, StringType())

df = df.withColumn(
    'antonyms', append_antonym_udf(col('vocab'))
)

df.show()

[Stage 2:>                                                          (0 + 1) / 1]

+-----------+-----------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      vocab|word_length|part_of_speech|         definitions|        sentences_eg|       derived_words|            synonyms|            antonyms|
+-----------+-----------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|     sahaja|          6|    kata tugas|1. yg satu-satuny...|1. tuan sahaja yg...|bersahaja;bersaha...|satu-satunya;cuma...|              jarang|
|      kerja|          5|     kata nama|1. usaha (kegiata...|1. beberapa mingg...|sekerja;bekerja;m...|khidmat;pekerjaan...|                    |
|       kaum|          4|     kata nama|1. puak, suku ban...|1. banyak kaum or...|   berkaum;perkauman|suku;kabilah;taef...|                    |
|      jelas|          5|      adjektif|1. terang (perkat...|1. suara itu tida...|berjelas-jelas;me...|terang;nyata;keta...|

## Calculate the Frequency of Usage from FIVE (5) Malay Essay

In [6]:
import requests
from bs4 import BeautifulSoup as bs
import string

urls = [
    "https://infopelajar.com.my/karangan-buli-siber-kesan-kepada-emosi/",
    "https://infopelajar.com.my/karangan-jenayah-siber/",
    "https://infopelajar.com.my/karangan-rumah-terbuka-di-malaysia/",
    "https://infopelajar.com.my/karangan-rasuah-ancaman-kepada-negara/",
    "https://infopelajar.com.my/contoh-karangan-pidato/"
]

url_contents = []
for url in urls:
    r = requests.get(url)
    soup = bs(r.content, 'html.parser')
        
    entry_content = soup.find('div', class_='entry-content')
    entry_content.find('div', class_='cta-box').decompose()
    essay_str = entry_content.get_text().replace('\n', ' ')
    essay_str = essay_str.translate(str.maketrans('', '', string.punctuation))

    url_contents.append(essay_str)

full_content_str = ' '.join(url_contents)
words_in_content = full_content_str.lower().split(' ')
        
while '' in words_in_content:
    words_in_content.remove('')

In [7]:
df_rows = df.collect()

freq_of_usage = []

for row in df_rows:
    count = 0
    vocab = row[0]

    for item in words_in_content:
        if item == vocab:
            count = count + 1

    freq_of_usage.append([vocab, count])

In [8]:
freq_df = spark.createDataFrame(freq_of_usage, ["vocab", "freq_of_usage"])

df = df.join(freq_df, on='vocab')
df.show()

+-----------+-----------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+
|      vocab|word_length|part_of_speech|         definitions|        sentences_eg|       derived_words|            synonyms|            antonyms|freq_of_usage|
+-----------+-----------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+
|     sahaja|          6|    kata tugas|1. yg satu-satuny...|1. tuan sahaja yg...|bersahaja;bersaha...|satu-satunya;cuma...|              jarang|           13|
|      kerja|          5|     kata nama|1. usaha (kegiata...|1. beberapa mingg...|sekerja;bekerja;m...|khidmat;pekerjaan...|                    |            0|
|       kaum|          4|     kata nama|1. puak, suku ban...|1. banyak kaum or...|   berkaum;perkauman|suku;kabilah;taef...|                    |            5|
|      jelas|          5|      adjektif|

## Assign Sentiment to each Word

In [9]:
from python_script.sentiment import *

In [10]:
import re

df_rows = df.collect()

sentiments = []

for rows in df_rows:
    vocab = rows[0]
    sentences = rows[4]
    
    trans_vocab = translateFromMyToEn(vocab)

    polarity = getPolarityOf(trans_vocab)
    if sentences != None:
        trans_sentences = []
        clean_sentences = re.sub('\d.', '', sentences)
        splitted_sentences = clean_sentences.split(';')
        
        for sente in splitted_sentences:
            trans_sentences.append(translateFromMyToEn(sente))
        
        total_subjectivity = 0
        sentences_count = 0
        for sente in trans_sentences:
            total_subjectivity = total_subjectivity + getSubjectivityValue(sente)
            sentences_count = sentences_count + 1
        
        subjectivity_score = total_subjectivity / sentences_count
        subjectivity = getSubjectivityFrom(subjectivity_score)
    else:
        subjectivity_score = getSubjectivityValue(trans_vocab)
        subjectivity = getSubjectivityOf(trans_vocab)
        
    sentiments.append([vocab, polarity, subjectivity, subjectivity_score])

In [11]:
sentiment_df = spark.createDataFrame(sentiments, ["vocab", "polarity", "subjectivity", "subjectivity_score"])

df = df.join(sentiment_df, on='vocab')
df = df.dropDuplicates()
df.show()

+-----------+-----------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------+--------------------+-------------------+
|      vocab|word_length|part_of_speech|         definitions|        sentences_eg|       derived_words|            synonyms|            antonyms|freq_of_usage|polarity|        subjectivity| subjectivity_score|
+-----------+-----------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------+--------------------+-------------------+
|    berniat|          7|     kata nama|1. bertujuan (aka...|1. aku pun tidak ...|berniat;berniat-n...|maksud;tujuan;ing...|                    |            0|    none|         fact-driven|                0.0|
|     mundur|          6|      adjektif|1. (berjalan, ber...|1. dia mundur set...|memundurkan;kemun...|merosot;meleset;m...|           maju;maju|            0| 

In [12]:
df = df.filter(df['vocab'] != 'kepentingan')
df.count()

219

## Write into CSV File

In [13]:
df.coalesce(1).write.csv('enriched_lexicon', header=True, mode='overwrite')

In [14]:
from python_script.hdfs_commander import *

run_hdfs_command('hdfs dfs -mv enriched_lexicon/part-00000-*.csv lexicon_data/enriched_lexicon.csv')
run_hdfs_command('hdfs dfs -rm -R enriched_lexicon')
run_hdfs_command('hdfs dfs -get lexicon_data/enriched_lexicon.csv lexicon_data/enriched_lexicon.csv')

Output: Deleted enriched_lexicon



## Terminate Spark Session

In [15]:
spark.stop()